# 01 — Master Equation Demo & Benchmark

**Aim.** Provide a reproducible notebook to (i) run the Backend‑1 master‑equation solver,
(ii) validate against analytic references in dense/sparse limits, and (iii) benchmark runtime and accuracy
vs. problem size and step size.

**Status.** This notebook is executable **as soon as** `MasterEquationSolver.evolve(...)` is implemented.
Until then, cells catch `NotImplementedError` and report a TODO.


In [ ]:
# Imports
import numpy as np
import time
import json
import math

import matplotlib.pyplot as plt

from unified_noise import ModelParams
from unified_noise.solver_me import MasterEquationSolver
from unified_noise.operators import harmonic_ops
from unified_noise.analytics import trace, n_expectation

# Matplotlib defaults (no specific colors to respect project's style constraints)
plt.rcParams['figure.figsize'] = (6.5, 4.0)
plt.rcParams['axes.grid'] = True

def hbar():
    return 1.054_571_817e-34


## Analytic Reference (Dense Gaussian Limit)

For a harmonic oscillator with frequency \(\omega\) weakly coupled to a thermal bath with mean occupancy \(\bar n\),
the thermal steady state satisfies
\[
\langle n \rangle_\mathrm{th} = \bar n,\quad
\langle x^2 \rangle_\mathrm{th} = \frac{\hbar}{2 m \omega}(2\bar n + 1),\quad
\langle p^2 \rangle_\mathrm{th} = \frac{\hbar m \omega}{2}(2\bar n + 1).
\]

We use these as accuracy targets when **jumps are disabled** (\(\lambda=0\)).


In [ ]:
def thermal_variances(m, omega, nbar):
    x2 = hbar() * (2.0*nbar + 1.0) / (2.0*m*omega)
    p2 = hbar() * m * omega * (2.0*nbar + 1.0) / 2.0
    return x2, p2


## Parameters

In [ ]:
# Physical parameters (adjust as needed)
m = 6.64e-26            # 40 amu [kg]
omega = 2*np.pi*2e6     # 2 MHz [rad/s]
gamma = 2*np.pi*5e3     # 5 kHz damping [1/s]

# Bath & jump parameters
nbar_bath = 0.05        # effective thermal occupancy for dense-limit check
kBT_eff = 4.1e-21       # ~ k_B T at room temperature (placeholder mapping)
lam = 0.0               # no jumps for dense-limit validation
jump_law = "gauss"
jump_pars = {"sigma_p": 1.0}

# Solver/truncation
N = 30
t_end = 2e-4            # total evolution time [s]
n_steps = 50
t_grid = np.linspace(0.0, t_end, n_steps+1)

params = ModelParams(
    m=m, omega=omega, gamma=gamma,
    kBT_eff=kBT_eff, lam=lam,
    jump_law=jump_law, jump_pars=jump_pars,
    nbar_bath=nbar_bath
)

# Initial state: vacuum in number basis
rho0 = np.zeros((N, N), dtype=complex)
rho0[0,0] = 1.0


## Build Solver (Backend‑1)

This uses the matrix‑free Liouvillian in Fock space. Once `evolve` is implemented,
the next cell performs time propagation.


In [ ]:
solver = MasterEquationSolver(N=N, params=params)

# cache number operator for ⟨n⟩
n_op, *_ = harmonic_ops(N, omega)


## Dense-Limit Validation (No Jumps)

We evolve \(\rho(t)\) with \(\lambda=0\) and compare the late-time moments to the thermal prediction.


In [ ]:
def run_dense_validation(solver, rho0, t_grid, n_op, params):
    try:
        traj = solver.evolve(rho0, t_grid)
    except NotImplementedError as e:
        print("TODO:", str(e))
        return None

    # Extract ⟨n⟩(t)
    n_expect = []
    for rho in traj:
        n_expect.append(n_expectation(rho, n_op))
    n_expect = np.asarray(n_expect)

    # Thermal targets (approximate mapping using nbar_bath)
    x2_ref, p2_ref = thermal_variances(params.m, params.omega, params.nbar_bath)

    return dict(n_of_t=n_expect, x2_ref=x2_ref, p2_ref=p2_ref, traj=traj)

dense_res = run_dense_validation(solver, rho0, t_grid, n_op, params)

if dense_res is not None:
    fig, ax = plt.subplots()
    ax.plot(t_grid[1:], dense_res['n_of_t'], label=r'$\langle n \rangle(t)$')
    ax.set_xlabel("time [s]")
    ax.set_ylabel(r"$\langle n \rangle$")
    ax.legend()
    plt.show()


## Sparse-Limit Benchmark (Renewal Heating)

Enable Poisson jumps (small \(\lambda\)) and verify the mean heating rate:
\[
\frac{d}{dt}\langle n \rangle \approx \lambda \langle \Delta n \rangle.
\]
We estimate the slope of \(\langle n \rangle(t)\) and compare with the prediction.


In [ ]:
def set_jumps(params, lam, jump_law="gauss", jump_pars=None):
    if jump_pars is None:
        jump_pars = {"sigma_p": 1.0}
    return ModelParams(
        m=params.m, omega=params.omega, gamma=params.gamma,
        kBT_eff=params.kBT_eff, lam=lam,
        jump_law=jump_law, jump_pars=jump_pars,
        nbar_bath=params.nbar_bath
    )

def expected_dn_from_dp_sigma(sigma_p, m, omega):
    # For small kicks, Δn ≈ (Δp)^2 / (2 m ħ ω); for Gaussian Δp~N(0,σ_p^2)
    # E[(Δp)^2] = σ_p^2 ⇒ E[Δn] ≈ σ_p^2 / (2 m ħ ω)
    return sigma_p**2 / (2.0 * m * hbar() * omega)

def estimate_slope(t, y):
    # simple linear fit slope dy/dt
    A = np.vstack([t, np.ones_like(t)]).T
    s, _ = np.linalg.lstsq(A, y, rcond=None)[0]
    return s

# Configure sparse jumps
lam_sparse = 0.5   # Hz
sigma_p = 1.5
params_sparse = set_jumps(params, lam=lam_sparse, jump_pars={"sigma_p": sigma_p})

solver_sparse = MasterEquationSolver(N=N, params=params_sparse)

try:
    traj_sparse = solver_sparse.evolve(rho0, t_grid)
    n_t = np.array([n_expectation(rho, n_op) for rho in traj_sparse])
    slope = estimate_slope(t_grid[1:], n_t)
    pred  = lam_sparse * expected_dn_from_dp_sigma(sigma_p, m, omega)
    print("Estimated d<n>/dt:", slope)
    print("Predicted  λ<E[Δn]>:", pred)
    fig, ax = plt.subplots()
    ax.plot(t_grid[1:], n_t, label=r'$\langle n \rangle(t)$')
    ax.set_xlabel("time [s]"); ax.set_ylabel(r"$\langle n \rangle$")
    ax.legend(); plt.show()
except NotImplementedError as e:
    print("TODO:", str(e))


## Runtime & Scaling

We measure wall-clock time for a single step vs. truncation size \(N\),
and (once implemented) vs. different time-step counts. This helps set default
parameters for production runs.


In [ ]:
def benchmark_runtime(N_list, t_grid, params, repeats=3):
    timings = []
    for N in N_list:
        solver = MasterEquationSolver(N=N, params=params)
        start = time.perf_counter()
        try:
            # run only first step to assess expm_multiply cost
            solver.evolve(np.eye(N, dtype=complex)/N, t_grid[:2])
        except NotImplementedError:
            # If not implemented yet, measure only construction overhead
            pass
        timings.append((N, (time.perf_counter() - start)))
    return np.array(timings)

N_list = [10, 20, 30, 40]
timings = benchmark_runtime(N_list, np.linspace(0, 1e-6, 2), params)

fig, ax = plt.subplots()
ax.plot(timings[:,0], timings[:,1], marker='o')
ax.set_xlabel("Fock truncation N")
ax.set_ylabel("Time per call [s] (construction/step)")
plt.show()


## Accuracy Summary (JSON)

We collect key metrics into a JSON blob for CI artifacts (optional).


In [ ]:
summary = {
    "N": int(N),
    "omega": float(omega),
    "gamma": float(gamma),
    "lam": float(lam),
    "nbar_bath": float(nbar_bath),
    "status": "pending_implementation"
}

print(json.dumps(summary, indent=2))
